In [ ]:
import pandas as pd
import numpy as np
from tkinter import *
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt 

In [ ]:
data=pd.read_csv('data.csv')
Breakfastdata=data['Breakfast']
BreakfastdataNumpy=Breakfastdata.to_numpy()
    
Lunchdata=data['Lunch']
LunchdataNumpy=Lunchdata.to_numpy()
    
Dinnerdata=data['Dinner']
DinnerdataNumpy=Dinnerdata.to_numpy()
Food_itemsdata=data['Food_items']

In [ ]:
data['BMI_Range'] = data['BMI_Range'].map({'Healthy': 3, 'Overweight': 4, 'Underweight': 1, 'Normal': 2})

In [ ]:
data['Veg/NonVeg'] = pd.to_numeric(data['Veg/NonVeg'], errors='coerce')

# Check for NaN values in 'Veg/NonVeg' column
nan_indices = data[data['Veg/NonVeg'].isna()].index

# Fill NaN values with a default value, such as 0 for non-veg
data['Veg/NonVeg'].fillna(0, inplace=True)

data['Veg/NonVeg'] = data['Veg/NonVeg'].astype(int)


data['Sugars'].fillna(data['Sugars'].mean(),inplace=True)
data['BMI_Range'].fillna(data['BMI_Range'].mean(),inplace=True)

In [ ]:
data.info()

In [ ]:
# data.drop(['BMI_Range'],axis=1,inplace=True)

In [ ]:


def separate_meal_items(data, BreakfastdataNumpy, LunchdataNumpy, DinnerdataNumpy, Food_itemsdata):
    # Initialize lists to store separated items and their IDs for each meal
    breakfastfoodseparated = []
    Lunchfoodseparated = []
    Dinnerfoodseparated = []
    
    breakfastfoodseparatedID = []
    LunchfoodseparatedID = []
    DinnerfoodseparatedID = []
    
    # Iterate through the data and separate items based on meal and Veg/NonVeg
    for i in range(len(data)):
        if BreakfastdataNumpy[i] == 1:
            breakfastfoodseparated.append(Food_itemsdata[i])
            breakfastfoodseparatedID.append(i)
        if LunchdataNumpy[i] == 1:
            Lunchfoodseparated.append(Food_itemsdata[i])
            LunchfoodseparatedID.append(i)
        if DinnerdataNumpy[i] == 1:
            Dinnerfoodseparated.append(Food_itemsdata[i])
            DinnerfoodseparatedID.append(i)
    
    # Retrieve rows by loc method for each meal
    LunchfoodseparatedIDdata = data.iloc[LunchfoodseparatedID].T.iloc[[0] + list(np.arange(4, 15))].T.to_numpy()
    breakfastfoodseparatedIDdata = data.iloc[breakfastfoodseparatedID].T.iloc[[0] + list(np.arange(4, 15))].T.to_numpy()
    DinnerfoodseparatedIDdata = data.iloc[DinnerfoodseparatedID].T.iloc[[0] + list(np.arange(4, 15))].T.to_numpy()

    return breakfastfoodseparatedIDdata, LunchfoodseparatedIDdata, DinnerfoodseparatedIDdata

 



In [ ]:
breakfastfoodseparatedIDdata, LunchfoodseparatedIDdata, DinnerfoodseparatedIDdata = separate_meal_items(data, BreakfastdataNumpy, LunchdataNumpy, DinnerdataNumpy, Food_itemsdata)

In [ ]:
breakfastfoodseparatedIDdata 

In [ ]:
def kmeans_clustering(data, num_clusters):
    # Extract the data for clustering
    Datacalorie = data[1:,1:len(data)]
    
    # Convert data to numpy array
    X = np.array(Datacalorie)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    
    # Get the cluster labels
    labels = kmeans.labels_
    
    return labels


In [ ]:
breakfast_labels = kmeans_clustering(breakfastfoodseparatedIDdata, 3)
lunch_labels= kmeans_clustering(LunchfoodseparatedIDdata, 3)
dinner_labels= kmeans_clustering(DinnerfoodseparatedIDdata, 3)


In [ ]:
breakfast_labels

## Recommendation without ml


In [ ]:
def calculate_bmr(weight, height, age, gender):
    if gender == 'M':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    elif gender == 'F':
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    return bmr

def user_bmr():
    # Get user input
    weight = float(input("Enter your weight in kg: "))
    height = float(input("Enter your height in cm: "))
    age = int(input("Enter your age in years: "))
    gender = input("Enter your gender (M/F): ")

    # Calculate BMR using the provided inputs
    bmr = calculate_bmr(weight, height, age, gender)

    return bmr


bmr_result = user_bmr()
print("Your Basal Metabolic Rate (BMR) is:", bmr_result)


In [ ]:
def calculate_calorie_requirement(bmr, activity_level):
    activity_factors = {'sedentary': 1.2, 'lightly_active': 1.375, 'moderately_active': 1.55, 'very_active': 1.725, 'extra_active': 1.9}
    activity_factor = activity_factors.get(activity_level, 1.2)  
    calorie_requirement = bmr * activity_factor
    return calorie_requirement

bmr = float(input("Enter your Basal Metabolic Rate (BMR): "))
activity_level = input("Enter your activity level (sedentary, lightly_active, moderately_active, very_active, extra_active): ")
calorie_requirement = calculate_calorie_requirement(bmr, activity_level)
calorie_requirement



In [ ]:
def recommend_all_foods(data, meal, veg_nonveg):
    if veg_nonveg.lower() not in ['v', 'nv']:
        raise ValueError("Invalid value for vegetarian/non-vegetarian. Use 'V' for vegetarian and 'NV' for non-vegetarian.")

    # Convert 'V' or 'NV' to 1 or 0, respectively
    veg_nonveg_code = 1 if veg_nonveg.lower() == 'v' else 0

    # Filter foods based on meal type and vegetarian/non-vegetarian preference
    foods_df = data[(data[meal] == 1) & (data['Veg/NonVeg'] == veg_nonveg_code)]

    # Convert DataFrame to a dictionary with separate entries for each food item
    foods_dict = {}
    for index, row in foods_df.iterrows():
        food_item = {
            'Meal': meal,
            'Veg/NonVeg': 'Vegetarian' if veg_nonveg_code == 1 else 'Non-Vegetarian',
            'Calories': row['Calories'],
            'Proteins': row['Proteins'],
            'Fats': row['Fats'],
            'Carbohydrates': row['Carbohydrates'],
            'Sugars': row['Sugars'],
            'VitaminD': row['VitaminD'],
            'Iron': row['Iron'],
            'Calcium': row['Calcium'],
            'Potassium': row['Potassium'],

            
        }
        foods_dict[row['Food_items']] = food_item

    return foods_dict

meal = input("Enter the meal type (Breakfast, Lunch, Dinner): ")
veg_nonveg = input("Are you vegetarian (V) or non-vegetarian (NV): ")

recommended_foods_dict = recommend_all_foods(data, meal, veg_nonveg)

print("Recommended Foods:")
for food_name, food_info in recommended_foods_dict.items():
    print(food_name)
    print(food_info)


In [ ]:
# Define the function to recommend foods based on the goal
def recommend_foods_for_goal(foods_dict, goal):
    if goal.lower() not in ['gain', 'loss', 'maintain']:
        raise ValueError("Invalid goal specified. Use 'gain', 'loss', or 'maintain'.")

    recommended_foods_calories = {}
    for food_name, food_info in foods_dict.items():
        calories = food_info['Calories']
        Proteins = food_info['Proteins']
        Fats = food_info['Fats']
        Carbs = food_info['Carbohydrates']
        if goal.lower() == 'gain':
            if (calories > 300 or
            Proteins > 25 or
            Fats > 20 or
            Carbs > 50) :
                recommended_foods_calories[food_name] = calories
        elif goal.lower() == 'loss':
            if (calories < 300 or
            Proteins < 20 or
            Fats < 10 or
            Carbs < 50):
                recommended_foods_calories[food_name] = calories
        else:  # For 'maintain'
            if (200 <= calories <= 400 or
            15 <= Proteins <= 25 or
            10 <= Fats <= 20 or
            40 <= Carbs <= 60):
                recommended_foods_calories[food_name] = calories

    return recommended_foods_calories

meal = input("Enter the meal type (Breakfast, Lunch, Dinner): ")
veg_nonveg = input("Are you vegetarian (V) or non-vegetarian (NV): ")

recommended_foods_dict = recommend_all_foods(data, meal, veg_nonveg)
# print("Recommended Foods:")
# for food_name, food_info in recommended_foods_dict.items():
#     print(food_name)
#     print(food_info)


goal = input("Enter your weight management goal (gain, loss, maintain): ")


recommended_foods_goal_calories = recommend_foods_for_goal(recommended_foods_dict, goal)

print("Recommended Foods for Goal with Calories:")
print(recommended_foods_goal_calories)


In [ ]:
def choose_food_items(data, calorie_requirement):
    chosen_items = []
    total_calories = 0
    
    while total_calories < calorie_requirement:
        print("Remaining Calorie Requirement:", calorie_requirement - total_calories)
        print("Choose from the following food items:")
        for food_name, food_info in data.items():
            print(f"{food_name}: Calories - {food_info['Calories']}")
        
        choice = input("Enter the name of the food item you want to add (or 'done' to finish): ")
        
        if choice.lower() == 'done':


            break
        
        if choice in data:
            chosen_items.append(choice)
            total_calories += data[choice]['Calories']
        else:
            print("Invalid food item. Please choose from the list.")
        
        print("Selected Food Items:", chosen_items)
    
    return chosen_items

calorie_requirement = int(input("Enter your calorie requirement: "))
chosen_food_items = choose_food_items(recommended_foods_dict, calorie_requirement)
print("Chosen Food Items:", chosen_food_items)
